In [8]:
from RNN import RNN 
from character_mapping import Character_Map 
import theano 
import theano.tensor as T
import numpy as np 


text_test = './../texts/melville.txt'
char_map_obj = Character_Map(text_test,'mapping.dat',overwrite=True, break_line=None)
char_map_obj.k_map()
x_, y_, shared_x, shared_y = char_map_obj.gen_x_and_y(filename=None)

# print(shared_y.get_value().shape[1])
nh = 100
nx = len(char_map_obj.unique_char)
ny = nx 

wx = theano.shared(name='wx',
                    value=0.2 * np.random.uniform(-1.0, 1.0,
                    (nx, nh))
                    .astype(theano.config.floatX)) #input weights

wh = theano.shared(name='wh',
                    value=0.2 * np.random.uniform(-1.0, 1.0,
                    (nh, nh))
                    .astype(theano.config.floatX)) #hidden layer weights

wy = theano.shared(name='wy',
                       value=0.2 * np.random.uniform(-1.0, 1.0,
                       (nh, ny))
                       .astype(theano.config.floatX)) #output weights

bh = theano.shared(name='bh',
                        value=np.zeros(nh,
                        dtype=theano.config.floatX)) #hidden layer bias

by = theano.shared(name='b',
                       value=np.zeros(ny,
                       dtype=theano.config.floatX)) #output layer bias

h0 = theano.shared(name='h0',
                        value=np.zeros(nh,
                        dtype=theano.config.floatX)) #initial h vector 




Time creating character mapping and pickling: 2.0342 sec
Time creating k map 3.124 sec
Time creating arrays: 3.078 sec


In [9]:
# print(shared_x.get_value()[0].shape)
# print(shared_y.get_value()[0].shape)

def step(x,h_tm1,wx,wh,wy,bh,by):
    h = T.tanh(T.dot(x,wx) + T.dot(h_tm1, wh) + bh)
    y_hat = by + T.dot(h,wy)
    y_guess = T.nnet.softmax(y_hat) 
    return h, y_guess
    
x = T.matrix('x')
y = T.imatrix('y')
# shared_y = T.cast(shared_y,'int32')

[h,s], _ = theano.scan(fn=step,
                    sequences=x,
                    outputs_info=[h0, None],
                    non_sequences = [wx,wh,wy,bh,by])


error = -T.mean(T.log(s)[T.arange(y.shape[0]), y])

fn = theano.function([x,y],error)

fn(shared_x.get_value()[0],shared_y.get_value()[0])

Elemwise{Cast{int32}}.0


AttributeError: 'TensorVariable' object has no attribute 'get_value'